In [7]:
pip install msoffcrypto-tool

Note: you may need to restart the kernel to use updated packages.


In [306]:
import pandas as pd
import io
import msoffcrypto as ex
import numpy as np
import warnings
import re

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)

#HL7Field = ['PID','ORC','OBR','OBX','NTE','SPM']
filedir = r"C:\Users\gmoye\FlatFileHL7\Extract_for_COVID_and_Influenza_20230503_1151.xlsx"
mapdir = r"C:\Users\gmoye\FlatFileHL7\FieldMap.xlsx"
hl7map = r"C:\Users\gmoye\FlatFileHL7\Maps.xlsx"
hometestmap = r"C:\Users\gmoye\FlatFileHL7\COVID-At-Home-Test-HL7-Guide-5-4-2022_0.xlsx"
pw = "test"

with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    decrypted_workbook = io.BytesIO()
    with open(filedir, 'rb') as file:
        office_file = ex.OfficeFile(file)
        office_file.load_key(password=pw)
        office_file.decrypt(decrypted_workbook)
    
    all_sheets = pd.read_excel(decrypted_workbook, sheet_name=None)
    sheets = list(all_sheets.keys())
    
    #print(sheets)
    df = pd.read_excel(decrypted_workbook, sheet_name=sheets[0], parse_dates=[['Order Date', 'Order Time'],['Result Date','Result Time'],['Collection Date','Collection Time']])
    
    df['record_no'] = np.arange(len(df))+1
    rcount = df.pop('record_no')
    df.insert(0, rcount.name, rcount)
    
    count = 0
    for col, vals in df[['Last Lab Results','Test Details']].items():
        count+=1
        count2 = 0
        for val in vals:
            count2+=1
            val = "'" + val + "'"
            val = "','".join(val.splitlines())
            val = str(val)
            #print(val)
            df[f'{col}'].loc[df['record_no'] == count2] = val
    
    df2 = df.loc[((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    df = df.loc[~((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    
    df = df.drop(columns=['record_no'])
    df2 = df2.drop(columns=['record_no'])
    
    df2preserve = df2.copy(deep=True)
    #display(df2preserve)
    df2 = df2[['Order ID', 'Last Lab Results','Test Details']]
    
    df2['record_no'] = np.arange(len(df2))+1
    rcount = df2.pop('record_no')
    df2.insert(0, rcount.name, rcount)
    
    for val in df2['record_no']:
        df2['record_no'].loc[df2['record_no'] == val] = str(val)+','+str(val)
        
    df3 = (df2.set_index(['Order ID'])).apply(lambda x: x.str.split(',').explode(['record_no','Last Lab Results','Test Details']))
    df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    
    dfsep = pd.merge(df2, df3, on = ['record_no'], how = 'inner')
    dfsep = dfsep.drop(columns=['Last Lab Results_x','Test Details_x'])
    df2preserve = df2preserve.drop(columns=['Last Lab Results','Test Details'])
    dfsep = dfsep.rename(columns={'Last Lab Results_y':'Last Lab Results','Test Details_y':'Test Details'})
    dfsep = pd.merge(dfsep, df2preserve, on = ['Order ID'], how = 'outer')
    
    df = pd.concat([df, dfsep]).reset_index(drop = True)
    
    df['Patient_ID_assigner'] = 'EPIC.ACC'
    df['Patient_ID_2_assigner'] = 'EPIC.MRN'
    df = df.rename(columns={'Order Date_Order Time':'Order_test_date','Result Date_Result Time':'Test_date','Collection Date_Collection Time':'Specimen_collection_date_time'})
    df = df.drop(columns=['record_no'])
    df.index.names = ['Field']
    
    column_names = df.columns.values.tolist()
    dfcolumns = pd.DataFrame(column_names)
    dfcolumns.rename(columns={0:'Field'}, inplace=True)
    
    dfmappings = pd.read_excel(mapdir)
    #display(dfmappings)
    dfmappings = dfmappings.loc[dfmappings['Not Needed'] != 'X']
    
    dfdistinctmappings = dfmappings[['Field','NatELR Field ID', 'Segment', 'Not Needed','Option-Set needed']]
    
    dfmerge = pd.merge(dfcolumns, dfdistinctmappings, on = ['Field'], how = 'outer')
    dfmerge['NatELR Field ID'].loc[dfmerge['NatELR Field ID'].isna()] = dfmerge['Field']
        
    vallist1 = []
    vallist2 = []
    for val in dfmerge['Field']:
        vallist1.append(val)
    for val in dfmerge['NatELR Field ID']:
        vallist2.append(val)
    #print(vallist1,'\n',vallist2,'\n')
    vallist1
    for values1 in vallist1:
        values1 = str(values1)
        for values2 in vallist2:
            values2 = str(values2)
            if values2 not in df.columns and values1 in df.columns and values1 != 'nan':
                #print(values2)
                df = df.rename(columns={f'{values1}':f'{values2}'})
            elif values1 == 'nan' and values2 != 'nan' and values2 not in df.columns:
                df[f'{values2}'] = None
        
    dfhl7 = pd.read_excel(hl7map)
    
    dfhl7home = pd.read_excel(hometestmap)
    
    HL7Line = dfhl7home['Segment'].unique()
    #print(HL7Line)
    
    dfhl7pivot = dfhl7.pivot(index=['Variable Name','Instructions'], columns=['Hardcode'], values=['HL7 Field'])#.reset_index(drop=False)
    dfhl7pivot.columns = dfhl7pivot.columns.droplevel(0)
    dfhl7pivot = dfhl7pivot.dropna(axis=1, how='all')
    dfhl7pivotreduction = dfhl7pivot.loc[:, dfhl7pivot.columns.notna()].dropna(axis=0, how='all').reset_index(drop=False)
    dfhl7pivotreduction["Variable Name"] = dfhl7pivotreduction["Variable Name"].fillna(dfhl7pivotreduction["Instructions"])
    dfhl7pivotreduction = dfhl7pivotreduction.set_index(["Instructions", "Variable Name"]).T#.reset_index(drop=False)
    dfhl7pivotreduction.columns = dfhl7pivotreduction.columns.droplevel(0)
    dfhl7pivotreduction = dfhl7pivotreduction.reset_index(drop=False)
    #display(dfhl7pivotreduction)
    lcol = list(dfhl7pivotreduction.columns)
    lcol.remove('Hardcode')
    dfhl7pivotreduction[lcol] = dfhl7pivotreduction[lcol].replace(regex=r"-\d{1,2}",value='').replace(regex=r"\.(.*)",value='')
    
    #display(dfdistinctmappings)
    #display(dfmappings)
    display(df)
    display(dfmerge)
    display(dfhl7pivotreduction)

,Testing_lab_name,Order_test_date,Test_date,Specimen_collection_date_time,Reporting_facility_name,Patient_ID,Patient_ID_2,Patient_last_name,Patient_first_name,Patient_middle_name,Patient_suffix,Patient_race,Patient_DOB,Patient_gender,Patient_ethnicity,Patient_street,Patient_street2,Street Address.1,Patient_city,Patient_county,Patient_state,Patient_zip_code,Patient_phone_number,Ordering_provider_ID,Ordering_provider_first_name,Ordering_provider_street,Ordering_provider_street2,Ordering_provider_city,Ordering_provider_state,Ordering_provider_zip_code,Ordering_provider_phone,Testing_lab_specimen_ID,Procedure,Ordered_test_code,Test_performed_number,Test_performed_description,Last Lab Results,Test_result_description,Specimen_type_code,Specimen_source_site_code,Comments,Patient_occupation,Employment Status,Patient Employer,Pregnant,Patient_ID_assigner,Patient_ID_2_assigner,Ordering_provider_last_name,Ordering_facility_name
Field,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2022-11-17 15:36:21,2022-11-17 15:59:12,2022-11-17 15:51:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N9171279,800390658,Ravnaas,Carol,Marlene,NaN,White or Caucasian,1939-04-11,Female,"Non-Hispanic, Latino/a, Latinx [1]",PO Box 192,NaN,TURTLE LAKE,1110,ND,58575,701-426-9366,1649839846,Kaitlyn R Wirtz,1177 Border Lane,NaN,Washburn,ND,58577,701-462-3396,347306692,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Negative','Negative',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN,None,None,None
1,2022-09-05 10:20:52,2022-09-05 10:32:44,2022-09-05 10:15:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N7050230,800535295,Baker,John,E,NaN,White or Caucasian,1953-04-17,Male,"Non-Hispanic, Latino/a, Latinx [1]",2ND AVE W,NaN,SAWYER,1812,ND,58781,701-509-4135,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,337986190,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Positive','Positive',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN,None,None,None
2,2022-06-21 08:16:18,2022-06-21 08:50:56,2022-06-21 08:20:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N4210361,800571449,Lemke,William,August,Jr.,White or Caucasian,1951-07-09,Male,"Non-Hispanic, Latino/a, Latinx [1]",PO Box 313,NaN,TURTLE LAKE,1110,ND,58575,701-448-2082,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,328550952,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Negative','Negative',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN,None,None,None
3,2022-09-06 08:32:19,2022-09-06 08:45:42,2022-09-06 08:41:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N7060385,800571449,Lemke,William,August,Jr.,White or Caucasian,1951-07-09,Male,"Non-Hispanic, Latino/a, Latinx [1]",PO Box 313,NaN,TURTLE LAKE,1110,ND,58575,701-448-2082,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,338673451,"SARS COV2 (TL, GAR. MIN & MAN ONLY) [LAB439701]",[94558-4],1,"SARS COV2 (TL, GAR. MIN & MAN ONLY)",'SARS CoV Ag: Positive','Positive',Nares,Nares,The SARS CoV Ag test has been performed under ...,NaN,Retired [5],NaN,No,EPIC.ACC,EPIC.MRN,None,None,None
4,2022-06-03 10:17:47,2022-06-03 10:26:37,2022-06-03 10:10:00,CHI ST ALEXIUS HEALTH - TURTLE LAKE [79],"NORTHERN PLAINS LABORATORY, LLC [78]",N4030558,800673696,Boyko,Maynard,NaN,NaN,White or Caucasian,1924-12-15,Male,"Non-Hispanic, Latino/a, Latinx [1]",1955 22nd St NW,NaN,RUSO,1110,ND,58778,701-679-2440,1235213695,Susan Gale Wardner,220 5th Ave W,NaN,TURTLE LAKE,ND,58575,701-448-2331,326122203,"

,Field,NatELR Field ID,Segment,Not Needed,Option-Set needed
0,Order_test_date,Order_test_date,NaN,NaN,NaN
1,Test_date,Test_date,NaN,NaN,NaN
2,Specimen_collection_date_time,Specimen_collection_date_time,NaN,NaN,NaN
3,Resulting Lab,Testing_lab_name,MSH,NaN,NaN
4,Resulting Agency,Reporting_facility_name,ORC,NaN,NaN
5,Acc #,Patient_ID,PID,NaN,NaN
6,MRN,Patient_ID_2,PID,NaN,NaN
7,Last Name,Patient_last_name,PID,NaN,NaN
8,First Name,Patient_first_name,PID,NaN,NaN
9,Patient Middle Name,Patient_middle_name,PID,NaN,NaN


Variable Name,Hardcode,MSH-10: Message Control ID,MSH-11: Processing ID,MSH-12: Version ID,MSH-15: Accept Acknowledgment Type,MSH-16: Application Acknowledgment Type,MSH-1: Field Separator,MSH-21: Message Profile Identifier,MSH-2: Encoding Characters,MSH-5: Receiving Application,MSH-6: Receiving Facility,MSH-9: Message Type,OBR-1: Set ID - OBR,ORC-1: Order Control,PID-1: Set ID - PID,PID-33: Last Update Date/Time,Date result released,Date this file was created,Format of the result,Name of the organization that created the patient ID,Name of the organization that created the patient ID #2,Order result status,Ordering Facility City,Ordering Facility State,Ordering facility county,Ordering facility street,Ordering facility zip code,Patient age units,Performing facility city,Performing facility county,Performing facility state,Performing facility street,Performing facility zip code,Report facil data source app,Reporting facility ID,Test performed code system,Test result status,Type of patient ID,Type of patient ID #2
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OBR,NaN,PID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0DSomeCLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSH,NaN,NaN,NaN,NaN
2,2.5.1,NaN,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORC,NaN,NaN,NaN,NaN,OBX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,401 North 9th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORC,NaN,NaN,NaN,NaN,NaN,OBX,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,58554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORC,NaN,NaN,NaN,NaN,NaN,OBX,NaN,NaN,NaN,NaN,NaN,NaN
6,AN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PID,NaN
7,Bismarck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORC,NaN,NaN,NaN,NaN,NaN,OBX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CHI.EPIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSH,NaN,NaN,NaN,NaN,NaN
9,EPIC.ACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
pd.set_option('display.max_rows', None)
dfhl7pivot = dfhl7.pivot(index=['Variable Name','Instructions'], columns=['Hardcode'], values=['HL7 Field'])#.reset_index(drop=False)
dfhl7pivot.columns = dfhl7pivot.columns.droplevel(0)
dfhl7pivot = dfhl7pivot.dropna(axis=1, how='all')
dfhl7pivotreduction = dfhl7pivot.loc[:, dfhl7pivot.columns.notna()].dropna(axis=0, how='all').reset_index(drop=False)
dfhl7pivotreduction["Variable Name"] = dfhl7pivotreduction["Variable Name"].fillna(dfhl7pivotreduction["Instructions"])
dfhl7pivotreduction = dfhl7pivotreduction.set_index(["Instructions", "Variable Name"]).T#.reset_index(drop=False)
dfhl7pivotreduction.columns = dfhl7pivotreduction.columns.droplevel(0)
dfhl7pivotreduction = dfhl7pivotreduction.reset_index(drop=False)

In [302]:
#display(dfmappings)
pd.set_option('display.max_rows', None)
#display(dfhl7)
dfhl7pivot = dfhl7.pivot(index=['Variable Name','Instructions'], columns=['Hardcode'], values=['HL7 Field'])#.reset_index(drop=False)
dfhl7pivot.columns = dfhl7pivot.columns.droplevel(0)
dfhl7pivot = dfhl7pivot.dropna(axis=1, how='all')
#display(dfhl7pivot)
dfhl7pivotreduction = dfhl7pivot.loc[:, dfhl7pivot.columns.notna()]
#display(dfhl7pivotreduction)
dfhl7pivotreduction = dfhl7pivotreduction.dropna(axis=0, how='all').reset_index(drop=False)
#display(dfhl7pivotreduction)
dfhl7pivotreduction["Variable Name"] = dfhl7pivotreduction["Variable Name"].fillna(dfhl7pivotreduction["Instructions"])
dfhl7pivotreduction = dfhl7pivotreduction.set_index(["Instructions", "Variable Name"]).T#.reset_index(drop=False)
dfhl7pivotreduction.columns = dfhl7pivotreduction.columns.droplevel(0)
dfhl7pivotreduction = dfhl7pivotreduction.reset_index(drop=False)
#display(dfhl7pivotreduction)
#display(dfhl7pivotreduction[list(dfhl7pivotreduction.columns)])
lcol = list(dfhl7pivotreduction.columns)
lcol.remove('Hardcode')
print(lcol)
#display(dfhl7pivotreduction)

['MSH-10: Message Control ID', 'MSH-11: Processing ID', 'MSH-12: Version ID', 'MSH-15: Accept Acknowledgment Type', 'MSH-16: Application Acknowledgment Type', 'MSH-1: Field Separator', 'MSH-21: Message Profile Identifier', 'MSH-2: Encoding Characters', 'MSH-5: Receiving Application', 'MSH-6: Receiving Facility', 'MSH-9: Message Type', 'OBR-1: Set ID - OBR', 'ORC-1: Order Control', 'PID-1: Set ID - PID', 'PID-33: Last Update Date/Time', 'Date result released', 'Date this file was created', 'Format of the result', 'Name of the organization that created the patient ID', 'Name of the organization that created the patient ID #2', 'Order result status', 'Ordering Facility City', 'Ordering Facility State', 'Ordering facility county', 'Ordering facility street', 'Ordering facility zip code', 'Patient age units', 'Performing facility city', 'Performing facility county', 'Performing facility state', 'Performing facility street', 'Performing facility zip code', 'Report facil data source app', 'Re